In [16]:
import compute_rhino3d
print(compute_rhino3d.__version__)


AttributeError: module 'compute_rhino3d' has no attribute '__version__'

In [ ]:
import os
import numpy as np
import rhino3dm
import open3d as o3d
from scipy.spatial import ConvexHull

# Function to load and process 3DM file efficientlyjjjjj
def load_3dm_model(input_file):
    model = rhino3dm.File3dm.Read(input_file)
    all_vertices = []
    all_faces = []
    
    for obj in model.Objects:
        geom = obj.Geometry

        if isinstance(geom, rhino3dm.Mesh):
            # ✅ Use NumPy directly to speed up processing
            vertices = np.array([[v.X, v.Y, v.Z] for v in geom.Vertices], dtype=np.float32)
            faces = np.array(geom.Faces, dtype=np.int32)  # Directly extract faces

            # ✅ Convert quads into triangles faster
            if faces.shape[1] == 4:  # If the mesh has quads
                tri_faces = np.hstack([faces[:, :3], faces[:, [0, 2, 3]]])
                faces = tri_faces.reshape(-1, 3)  # Reshape into triangles

            all_vertices.append(vertices)
            all_faces.append(faces)

    if not all_vertices:
        print("❌ No valid mesh found in 3DM file.")
        return None, None

    # ✅ Use NumPy stack for fast merging
    combined_vertices = np.vstack(all_vertices)
    combined_faces = np.vstack(all_faces)
    
    return combined_vertices, combined_faces

# Function to analyze and visualize zoning
def analyze_and_visualize_zoning(vertices, faces):
    min_bounds = np.min(vertices, axis=0)
    max_bounds = np.max(vertices, axis=0)
    min_z = min_bounds[2]
    max_z = max_bounds[2]

    # Step 1: Detect Core Area (Lowest 20% of building)
    core_mask = vertices[:, 2] < min_z + (max_z - min_z) * 0.2

    # Step 2: Detect Circulation Paths using Convex Hull
    hull = ConvexHull(vertices[:, :2])  
    circulation_mask = np.zeros(len(vertices), dtype=bool)
    circulation_mask[hull.vertices] = True  # Mark circulation as blue

    # Step 3: Assign Colors (Core = Red, Circulation = Blue, Default = White)
    colors = np.ones((len(vertices), 3), dtype=np.float32)
    colors[core_mask] = [1, 0, 0]  # Red for core
    colors[circulation_mask] = [0, 0, 1]  # Blue for circulation

    # Create Open3D mesh for visualization
    o3d_mesh = o3d.geometry.TriangleMesh()
    o3d_mesh.vertices = o3d.utility.Vector3dVector(vertices)
    o3d_mesh.triangles = o3d.utility.Vector3iVector(faces)
    o3d_mesh.vertex_colors = o3d.utility.Vector3dVector(colors)

    # Show mesh in Open3D
    o3d.visualization.draw_geometries([o3d_mesh])

# Function to analyze basic massing geometry
def analyze_geometry(vertices):
    min_bounds = np.min(vertices, axis=0)
    max_bounds = np.max(vertices, axis=0)
    footprint_area = (max_bounds[0] - min_bounds[0]) * (max_bounds[1] - min_bounds[1])
    height = max_bounds[2] - min_bounds[2]

    print("\n--- Massing Analysis ---")
    print(f"Footprint Area: {footprint_area:.2f} sq.m")
    print(f"Total Height: {height:.2f} m")

# Example Usage
input_file = r"C:\Users\papad\Desktop\IAAC\2ndTerm\Studio2\basicformMETERSmesh.3dm"  # Replace with actual file path

vertices, faces = load_3dm_model(input_file)

if vertices is not None and faces is not None:
    analyze_geometry(vertices)
    analyze_and_visualize_zoning(vertices, faces)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [26]:
import os
import numpy as np
import rhino3dm
import open3d as o3d
from scipy.spatial import ConvexHull

# Function to load and process 3DM file
def load_3dm_model(input_file):
    print(f"🔄 Loading file: {input_file}")

    model = rhino3dm.File3dm.Read(input_file)
    print(f"🔄 Found {len(model.Objects)} objects in the file")

    vertices_list = []
    faces_list = []
    
    for obj in model.Objects:
        geom = obj.Geometry

        # Convert Rhino Mesh directly
        if isinstance(geom, rhino3dm.Mesh):
            vertices = np.array([[v.X, v.Y, v.Z] for v in geom.Vertices])
            
            # ✅ Speed up face extraction
            faces = np.array(geom.Faces, dtype=np.int32)  

            vertices_list.append(vertices)
            faces_list.append(faces)

    if not vertices_list:
        print("❌ No valid mesh found in 3DM file.")
        return None, None

    combined_vertices = np.vstack(vertices_list)
    combined_faces = np.vstack(faces_list)

    print(f"✅ Loaded {len(combined_vertices)} vertices and {len(combined_faces)} faces")
    return combined_vertices, combined_faces

# Function to analyze and visualize zoning
def analyze_and_visualize_zoning(vertices, faces):
    print("🔄 Starting zoning analysis...")

    # Compute bounding box
    min_bounds = np.min(vertices, axis=0)
    max_bounds = np.max(vertices, axis=0)
    min_x, min_y, min_z = min_bounds
    max_x, max_y, max_z = max_bounds

    print("📌 Bounding Box Calculated")
    
    # Step 1: Detect Core Area (Lowest 20% of building)
    core_z_threshold = min_z + (max_z - min_z) * 0.2
    core_mask = vertices[:, 2] < core_z_threshold
    print(f"📌 Core Area Detected: {np.sum(core_mask)} points")

    # Step 2: Detect Circulation Paths using Convex Hull
    print(f"🔄 Running Convex Hull on {len(vertices)} points...")
    if len(vertices) > 5000:
        print("⚠️ Large model detected! Simplifying circulation detection...")
        sampled_vertices = vertices[np.random.choice(len(vertices), 5000, replace=False)]
        hull = ConvexHull(sampled_vertices[:, :2])  
    else:
        try:
            hull = ConvexHull(vertices[:, :2])  
        except Exception as e:
            print(f"❌ Convex Hull Failed: {e}")
            return

    print(f"📌 Circulation Hull Detected: {len(hull.vertices)} points")
    
    circulation_mask = np.zeros(len(vertices), dtype=bool)
    circulation_mask[hull.vertices] = True  # Mark circulation as blue

    # Step 3: Assign Colors (Core = Red, Circulation = Blue, Façade = Green)
    colors = np.ones((len(vertices), 3))  # Default white
    colors[core_mask] = [1, 0, 0]  # Red for core areas
    colors[circulation_mask] = [0, 0, 1]  # Blue for circulation

    print("✅ Zoning analysis complete!")

    # Create a visual mesh with color information
    o3d_mesh = o3d.geometry.TriangleMesh()
    o3d_mesh.vertices = o3d.utility.Vector3dVector(vertices)
    o3d_mesh.triangles = o3d.utility.Vector3iVector(faces)
    o3d_mesh.vertex_colors = o3d.utility.Vector3dVector(colors)

    print("🔄 Starting visualization...")
    o3d.visualization.draw_geometries([o3d_mesh])
    print("✅ Visualization complete!")

# Function to analyze basic massing geometry
def analyze_geometry(vertices):
    print("🔄 Starting massing analysis...")

    # Compute bounding box
    min_bounds = np.min(vertices, axis=0)
    max_bounds = np.max(vertices, axis=0)
    footprint_area = (max_bounds[0] - min_bounds[0]) * (max_bounds[1] - min_bounds[1])
    height = max_bounds[2] - min_bounds[2]

    print("\n--- Massing Analysis ---")
    print(f"Footprint Area: {footprint_area:.2f} sq.m")
    print(f"Total Height: {height:.2f} m")
    print("✅ Massing analysis complete!")

# Example Usage
input_file = r"C:\Users\papad\Desktop\IAAC\2ndTerm\Studio2\basicformMETERSmesh.3dm"  # Replace with actual file path

vertices, faces = load_3dm_model(input_file)

if vertices is not None and faces is not None:
    analyze_geometry(vertices)
    analyze_and_visualize_zoning(vertices, faces)


🔄 Loading file: C:\Users\papad\Desktop\IAAC\2ndTerm\Studio2\basicformMETERSmesh.3dm
🔄 Found 68 objects in the file
✅ Loaded 988 vertices and 900 faces
🔄 Starting massing analysis...

--- Massing Analysis ---
Footprint Area: 36.00 sq.m
Total Height: 6.00 m
✅ Massing analysis complete!
🔄 Starting zoning analysis...
📌 Bounding Box Calculated
📌 Core Area Detected: 415 points
🔄 Running Convex Hull on 988 points...
📌 Circulation Hull Detected: 51 points
✅ Zoning analysis complete!


RuntimeError: 

In [ ]:
import os
import numpy as np
import rhino3dm
import open3d as o3d
from scipy.spatial import ConvexHull

# Function to load and process 3DM file
def load_3dm_model(input_file):
    print(f"🔄 Loading file: {input_file}")

    model = rhino3dm.File3dm.Read(input_file)
    print(f"🔄 Found {len(model.Objects)} objects in the file")

    vertices_list = []
    faces_list = []
    
    for obj in model.Objects:
        geom = obj.Geometry

        # Convert Rhino Mesh directly
        if isinstance(geom, rhino3dm.Mesh):
            vertices = np.array([[v.X, v.Y, v.Z] for v in geom.Vertices])
            faces_raw = np.array(geom.Faces, dtype=np.int32)  # Raw faces
            
            # ✅ Fix: Convert quads (4-sided faces) to triangles (3-sided faces)
            faces = []
            for face in faces_raw:
                if len(face) == 4:  # Quad face
                    faces.append([face[0], face[1], face[2]])  # First triangle
                    faces.append([face[0], face[2], face[3]])  # Second triangle
                elif len(face) == 3:  # Already a triangle
                    faces.append(face)
                else:
                    print(f"⚠️ Unexpected face format: {face}")  # Debugging

            faces = np.array(faces, dtype=np.int32)  # Convert to NumPy array
            
            # ✅ Ensure all face indices are valid
            faces = faces[np.all(faces < len(vertices), axis=1)]

            vertices_list.append(vertices)
            faces_list.append(faces)

    if not vertices_list:
        print("❌ No valid mesh found in 3DM file.")
        return None, None

    combined_vertices = np.vstack(vertices_list)
    combined_faces = np.vstack(faces_list)

    print(f"✅ Loaded {len(combined_vertices)} vertices and {len(combined_faces)} faces")
    return combined_vertices, combined_faces

# Function to analyze and visualize zoning
def analyze_and_visualize_zoning(vertices, faces):
    print("🔄 Starting zoning analysis...")

    # Compute bounding box
    min_bounds = np.min(vertices, axis=0)
    max_bounds = np.max(vertices, axis=0)
    min_x, min_y, min_z = min_bounds
    max_x, max_y, max_z = max_bounds

    print("📌 Bounding Box Calculated")
    
    # Step 1: Detect Core Area (Lowest 20% of building)
    core_z_threshold = min_z + (max_z - min_z) * 0.2
    core_mask = vertices[:, 2] < core_z_threshold
    print(f"📌 Core Area Detected: {np.sum(core_mask)} points")

    # Step 2: Detect Circulation Paths using Convex Hull
    print(f"🔄 Running Convex Hull on {len(vertices)} points...")
    try:
        hull = ConvexHull(vertices[:, :2])  
    except Exception as e:
        print(f"❌ Convex Hull Failed: {e}")
        return

    print(f"📌 Circulation Hull Detected: {len(hull.vertices)} points")
    
    circulation_mask = np.zeros(len(vertices), dtype=bool)
    circulation_mask[hull.vertices] = True  # Mark circulation as blue

    # Step 3: Assign Colors (Core = Red, Circulation = Blue, Façade = Green)
    colors = np.ones((len(vertices), 3))  # Default white
    colors[core_mask] = [1, 0, 0]  # Red for core areas
    colors[circulation_mask] = [0, 0, 1]  # Blue for circulation

    print("✅ Zoning analysis complete!")

    # ✅ Fix: Check if faces are empty before visualization
    if len(faces) == 0:
        print("❌ No valid faces to display! Skipping visualization.")
        return

    # ✅ Debug: Print sample face data
    print("🔍 Face data sample:", faces[:5])
    print("🔍 Min/Max face index:", np.min(faces), np.max(faces))

    # Create a visual mesh with color information
    o3d_mesh = o3d.geometry.TriangleMesh()
    o3d_mesh.vertices = o3d.utility.Vector3dVector(vertices)
    o3d_mesh.triangles = o3d.utility.Vector3iVector(faces)
    o3d_mesh.vertex_colors = o3d.utility.Vector3dVector(colors)

    print("🔄 Starting visualization...")
    o3d.visualization.draw_geometries([o3d_mesh])
    print("✅ Visualization complete!")

# Function to analyze basic massing geometry
def analyze_geometry(vertices):
    print("🔄 Starting massing analysis...")

    # Compute bounding box
    min_bounds = np.min(vertices, axis=0)
    max_bounds = np.max(vertices, axis=0)
    footprint_area = (max_bounds[0] - min_bounds[0]) * (max_bounds[1] - min_bounds[1])
    height = max_bounds[2] - min_bounds[2]

    print("\n--- Massing Analysis ---")
    print(f"Footprint Area: {footprint_area:.2f} sq.m")
    print(f"Total Height: {height:.2f} m")
    print("✅ Massing analysis complete!")

# Example Usage
input_file = r"C:\Users\papad\Desktop\IAAC\2ndTerm\Studio2\basicformMETERSmesh.3dm"  # Replace with actual file path

vertices, faces = load_3dm_model(input_file)

if vertices is not None and faces is not None:
    analyze_geometry(vertices)
    analyze_and_visualize_zoning(vertices, faces)


In [ ]:
import os
import numpy as np
import rhino3dm
import open3d as o3d
from scipy.spatial import ConvexHull

# Function to load and process 3DM file
def load_3dm_model(input_file):
    print(f"🔄 Loading file: {input_file}")

    model = rhino3dm.File3dm.Read(input_file)
    print(f"🔄 Found {len(model.Objects)} objects in the file")

    vertices_list = []
    faces_list = []
    
    for obj in model.Objects:
        geom = obj.Geometry

        # Convert Rhino Mesh directly
        if isinstance(geom, rhino3dm.Mesh):
            vertices = np.array([[v.X, v.Y, v.Z] for v in geom.Vertices])
            faces_raw = np.array(geom.Faces, dtype=np.int32)  # Raw faces
            
            # ✅ Fix: Convert quads (4-sided faces) to triangles (3-sided faces)
            faces = []
            for face in faces_raw:
                if len(face) == 4:  # Quad face
                    faces.append([face[0], face[1], face[2]])  # First triangle
                    faces.append([face[0], face[2], face[3]])  # Second triangle
                elif len(face) == 3:  # Already a triangle
                    faces.append(face)
                else:
                    print(f"⚠️ Unexpected face format: {face}")  # Debugging

            faces = np.array(faces, dtype=np.int32)  # Convert to NumPy array
            
            # ✅ Ensure all face indices are valid
            faces = faces[np.all(faces < len(vertices), axis=1)]

            vertices_list.append(vertices)
            faces_list.append(faces)

    if not vertices_list:
        print("❌ No valid mesh found in 3DM file.")
        return None, None

    combined_vertices = np.vstack(vertices_list)
    combined_faces = np.vstack(faces_list)

    print(f"✅ Loaded {len(combined_vertices)} vertices and {len(combined_faces)} faces")
    return combined_vertices, combined_faces

# Function to analyze and visualize zoning
def analyze_and_visualize_zoning(vertices, faces):
    print("🔄 Starting zoning analysis...")

    # Compute bounding box
    min_bounds = np.min(vertices, axis=0)
    max_bounds = np.max(vertices, axis=0)
    min_x, min_y, min_z = min_bounds
    max_x, max_y, max_z = max_bounds

    print("📌 Bounding Box Calculated")
    
    # Step 1: Detect Core Area (Lowest 20% of building)
    core_z_threshold = min_z + (max_z - min_z) * 0.2
    core_mask = vertices[:, 2] < core_z_threshold
    print(f"📌 Core Area Detected: {np.sum(core_mask)} points")

    # Step 2: Detect Circulation Paths using Convex Hull
    print(f"🔄 Running Convex Hull on {len(vertices)} points...")
    try:
        hull = ConvexHull(vertices[:, :2])  
    except Exception as e:
        print(f"❌ Convex Hull Failed: {e}")
        return

    print(f"📌 Circulation Hull Detected: {len(hull.vertices)} points")
    
    circulation_mask = np.zeros(len(vertices), dtype=bool)
    circulation_mask[hull.vertices] = True  # Mark circulation as blue

    # Step 3: Assign Colors (Core = Red, Circulation = Blue, Façade = Green)
    colors = np.ones((len(vertices), 3))  # Default white
    colors[core_mask] = [1, 0, 0]  # Red for core areas
    colors[circulation_mask] = [0, 0, 1]  # Blue for circulation

    print("✅ Zoning analysis complete!")

    # ✅ Fix: Check if faces are empty before visualization
    if len(faces) == 0:
        print("❌ No valid faces to display! Skipping visualization.")
        return

    # ✅ Debug: Print sample face data
    print("🔍 Face data sample:", faces[:5])
    print("🔍 Min/Max face index:", np.min(faces), np.max(faces))

    # Create a visual mesh with color information
    o3d_mesh = o3d.geometry.TriangleMesh()
    o3d_mesh.vertices = o3d.utility.Vector3dVector(vertices)
    o3d_mesh.triangles = o3d.utility.Vector3iVector(faces)
    o3d_mesh.vertex_colors = o3d.utility.Vector3dVector(colors)

    print("🔄 Starting visualization...")
    o3d.visualization.draw_geometries([o3d_mesh])
    print("✅ Visualization complete!")

# Function to analyze basic massing geometry
def analyze_geometry(vertices):
    print("🔄 Starting massing analysis...")

    # Compute bounding box
    min_bounds = np.min(vertices, axis=0)
    max_bounds = np.max(vertices, axis=0)
    footprint_area = (max_bounds[0] - min_bounds[0]) * (max_bounds[1] - min_bounds[1])
    height = max_bounds[2] - min_bounds[2]

    print("\n--- Massing Analysis ---")
    print(f"Footprint Area: {footprint_area:.2f} sq.m")
    print(f"Total Height: {height:.2f} m")
    print("✅ Massing analysis complete!")

# Example Usage
input_file = r"C:\Users\papad\Desktop\IAAC\2ndTerm\Studio2\basicformMETERSmesh.3dm"  # Replace with actual file path

vertices, faces = load_3dm_model(input_file)

if vertices is not None and faces is not None:
    analyze_geometry(vertices)
    analyze_and_visualize_zoning(vertices, faces)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [1]:
import os
import numpy as np
import rhino3dm
import open3d as o3d
from scipy.spatial import ConvexHull

print("✅ Script started successfully!")

# Check if the file exists
input_file = r"C:\Users\papad\Desktop\IAAC\2ndTerm\Studio2\basicformMETERSmesh.3dm"
if not os.path.exists(input_file):
    print(f"❌ File not found: {input_file}")
    exit()

# Function to load and process 3DM file
def load_3dm_model(input_file):
    print(f"🔄 Loading file: {input_file}")

    model = rhino3dm.File3dm.Read(input_file)
    if model is None:
        raise ValueError("❌ Rhino3DM failed to read the file!")

    print(f"🔄 Found {len(model.Objects)} objects in the file")

    vertices_list = []
    faces_list = []
    
    for obj in model.Objects:
        geom = obj.Geometry
        print("🔄 Checking model geometry...")

        if isinstance(geom, rhino3dm.Mesh):
            vertices = np.array([[v.X, v.Y, v.Z] for v in geom.Vertices])
            faces_raw = np.array(geom.Faces, dtype=np.int32)

            print(f"🔄 Extracting {len(vertices)} vertices and {len(faces_raw)} faces...")

            faces = []
            for face in faces_raw:
                if len(face) == 4:
                    faces.append([face[0], face[1], face[2]])
                    faces.append([face[0], face[2], face[3]])
                elif len(face) == 3:
                    faces.append(face)
                else:
                    print(f"⚠️ Unexpected face format: {face}")

            faces = np.array(faces, dtype=np.int32)
            faces = faces[np.all(faces < len(vertices), axis=1)]

            vertices_list.append(vertices)
            faces_list.append(faces)

    if not vertices_list:
        print("❌ No valid mesh found in 3DM file.")
        return None, None

    combined_vertices = np.vstack(vertices_list)
    combined_faces = np.vstack(faces_list)

    print(f"✅ Loaded {len(combined_vertices)} vertices and {len(combined_faces)} faces")
    return combined_vertices, combined_faces

# Example Usage
vertices, faces = load_3dm_model(input_file)

if vertices is not None and faces is not None:
    print("✅ Successfully loaded the model!")
else:
    print("❌ Model loading failed.")


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
✅ Script started successfully!
🔄 Loading file: C:\Users\papad\Desktop\IAAC\2ndTerm\Studio2\basicformMETERSmesh.3dm
🔄 Found 68 objects in the file
🔄 Checking model geometry...
🔄 Checking model geometry...
🔄 Checking model geometry...
🔄 Checking model geometry...
🔄 Checking model geometry...
🔄 Checking model geometry...
🔄 Checking model geometry...
🔄 Checking model geometry...
🔄 Checking model geometry...
🔄 Checking model geometry...
🔄 Checking model geometry...
🔄 Checking model geometry...
🔄 Checking model geometry...
🔄 Checking model geometry...
🔄 Checking model geometry...
🔄 Checking model geometry...
🔄 Checking model geometry...
🔄 Checking model geometry...
🔄 Checking model geometry...
🔄 Checking model geometry...
🔄 Checking model geometry...
🔄 Checking model geometry...
🔄 Checking model geometry...
🔄 Checking model g